### In this notebook we will do some exploratory data analysis on our dataset.

In [1]:
import pandas as pd
import numpy as np
import copy
from sklearn.decomposition import PCA
import statsmodels.api as sm

In [2]:
ogdf = pd.read_csv('./final_dataframes/main_training_dataframe.csv')

In [3]:
ogdf

,Unnamed: 0,time,temperature_2m_1,relative_humidity_2m_1,wind_speed_10m_1,wind_direction_10m_1,temperature_2m_33,relative_humidity_2m_33,wind_speed_10m_33,wind_direction_10m_33,...,relative_humidity_2m_34,wind_speed_10m_34,wind_direction_10m_34,temperature_2m_38,relative_humidity_2m_38,wind_speed_10m_38,wind_direction_10m_38,Wind,Year,MonthDay
0,0,2019-01-01 00:00:00,-7.4,81,17.9,146,-5.7,87,12.4,100,...,88,13.0,132,-5.6,84,18.8,144,1933.0,2019,01-01
1,1,2019-01-01 01:00:00,-7.0,80,18.9,145,-5.8,87,13.6,102,...,87,14.6,130,-5.8,84,19.2,142,1996.0,2019,01-01
2,2,2019-01-01 02:00:00,-7.1,80,21.3,146,-5.9,88,12.9,103,...,89,16.0,121,-5.8,83,17.9,142,1936.0,2019,01-01
3,3,2019-01-01 03:00:00,-6.8,80,22.2,148,-5.9,90,15.9,103,...,87,16.2,122,-6.0,84,15.0,136,2023.0,2019,01-01
4,4,2019-01-01 04:00:00,-6.8,81,22.4,143,-5.9,90,15.6,109,...,86,16.1,114,-6.3,86,15.8,131,2147.0,2019,01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35059,2022-12-31 19:00:00,3.1,92,7.7,217,1.1,94,5.6,45,...,97,4.9,234,1.8,98,4.6,198,145.0,2022,12-31
35060,35060,2022-12-31 20:00:00,2.6,97,2.5,188,1.0,95,4.6,51,...,98,1.1,90,2.2,100,5.1,188,118.0,2022,12-31
35061,35061,2022-12-31 21:00:00,2.9,96,2.7,203,0.8,96,6.0,17,...,99,1.8,90,3.4,99,2.2,189,31.0,2022,12-31
35062,35062,2022-12-31 22:00:00,3.8,94,3.1,54,0.6,96,5.2,335,...,99,3.3,96,3.0,100,1.8,127,33.0,2022,12-31


In [4]:
ogdf.columns

Index(['Unnamed: 0', 'time', 'temperature_2m_1', 'relative_humidity_2m_1',
       'wind_speed_10m_1', 'wind_direction_10m_1', 'temperature_2m_33',
       'relative_humidity_2m_33', 'wind_speed_10m_33', 'wind_direction_10m_33',
       ...
       'relative_humidity_2m_34', 'wind_speed_10m_34', 'wind_direction_10m_34',
       'temperature_2m_38', 'relative_humidity_2m_38', 'wind_speed_10m_38',
       'wind_direction_10m_38', 'Wind', 'Year', 'MonthDay'],
      dtype='object', length=161)

I want to do some PCA to start. What variable am I interested in? Let $t$ be our time variable given in hours. Given wheather data from time $t=t_0$ to time $t=t_1$ I am interested in the sum of the windpower generated from $t=t_0+1$ to $t+24$. I would like to compare this to my weather data. Let's make a CSV with the follows: remove Year and Monthday. Let's remove time and instead use the "Unnamed:0" column to count hours, starting from the beginning of the year in 2019.

In [5]:
dayoutdf = copy.deepcopy(ogdf)

In [6]:
dayoutdf = dayoutdf.drop(columns=['time','Year', 'MonthDay'])
dayoutdf = dayoutdf.rename(columns={'Unnamed: 0': 't_hours'})

In [7]:
#experiment cell
data = {'values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
df = pd.DataFrame(data)

# Calculate the rolling mean with a window of 3
rolling_mean = df['values'].rolling(window=3).sum().shift(-3)
print(rolling_mean)

0     9.0
1    12.0
2    15.0
3    18.0
4    21.0
5    24.0
6    27.0
7     NaN
8     NaN
9     NaN
Name: values, dtype: float64


In [8]:
#now we need to take the entries in 'Wind' and then add 24 of them together
n=24
dayoutdf['Wind'] = dayoutdf['Wind'].rolling(window=n).sum().shift(-n) #this replaces 'Wind' time t with the sum of the entries in 'Wind' at t+1,... t+24

In [9]:
dayoutdf = dayoutdf.rename(columns={'Wind': 'Day_Ahead_Power'})

In [10]:
dayoutdf

,t_hours,temperature_2m_1,relative_humidity_2m_1,wind_speed_10m_1,wind_direction_10m_1,temperature_2m_33,relative_humidity_2m_33,wind_speed_10m_33,wind_direction_10m_33,temperature_2m_2,...,wind_direction_10m_31,temperature_2m_34,relative_humidity_2m_34,wind_speed_10m_34,wind_direction_10m_34,temperature_2m_38,relative_humidity_2m_38,wind_speed_10m_38,wind_direction_10m_38,Day_Ahead_Power
0,0,-7.4,81,17.9,146,-5.7,87,12.4,100,-13.4,...,145,-11.1,88,13.0,132,-5.6,84,18.8,144,53069.0
1,1,-7.0,80,18.9,145,-5.8,87,13.6,102,-13.6,...,147,-10.4,87,14.6,130,-5.8,84,19.2,142,53355.0
2,2,-7.1,80,21.3,146,-5.9,88,12.9,103,-14.1,...,136,-10.9,89,16.0,121,-5.8,83,17.9,142,53663.0
3,3,-6.8,80,22.2,148,-5.9,90,15.9,103,-12.8,...,128,-10.0,87,16.2,122,-6.0,84,15.0,136,53689.0
4,4,-6.8,81,22.4,143,-5.9,90,15.6,109,-11.9,...,129,-9.4,86,16.1,114,-6.3,86,15.8,131,53600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35059,3.1,92,7.7,217,1.1,94,5.6,45,-2.5,...,260,1.0,97,4.9,234,1.8,98,4.6,198,NaN
35060,35060,2.6,97,2.5,188,1.0,95,4.6,51,-1.1,...,259,1.9,98,1.1,90,2.2,100,5.1,188,NaN
35061,35061,2.9,96,2.7,203,0.8,96,6.0,17,-0.9,...,270,1.6,99,1.8,90,3.4,99,2.2,189,NaN
35062,35062,3.8,94,3.1,54,0.6,96,5.2,335,3.0,...,45,1.4,99,3.3,96,3.0,100,1.8,127,NaN


In [11]:
#making this df a matrix
dayoutarray = dayoutdf.to_numpy()

## Descriptive Stats

In [12]:
ogdf.describe()

,Unnamed: 0,temperature_2m_1,relative_humidity_2m_1,wind_speed_10m_1,wind_direction_10m_1,temperature_2m_33,relative_humidity_2m_33,wind_speed_10m_33,wind_direction_10m_33,temperature_2m_2,...,temperature_2m_34,relative_humidity_2m_34,wind_speed_10m_34,wind_direction_10m_34,temperature_2m_38,relative_humidity_2m_38,wind_speed_10m_38,wind_direction_10m_38,Wind,Year
count,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,...,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000
mean,17531.500000,4.681411,73.201346,14.389191,207.287874,4.069296,72.305441,13.361519,216.556097,1.944556,...,2.556263,76.076146,12.748138,218.991302,4.008750,72.276523,12.651241,218.051820,1303.409965,2020.499658
std,10122.249256,11.594230,14.859160,6.929509,88.373778,12.853228,14.483281,7.244159,86.048975,11.573345,...,11.778780,16.236344,6.274645,95.029055,12.232864,15.215812,6.258694,86.492533,829.374476,1.117744
min,0.000000,-27.700000,23.000000,0.000000,1.000000,-34.000000,20.000000,0.000000,1.000000,-31.200000,...,-31.400000,11.000000,0.000000,1.000000,-29.200000,21.000000,0.000000,1.000000,-12.000000,2019.000000
25%,8765.750000,-4.100000,63.000000,9.300000,152.000000,-5.300000,62.000000,7.900000,140.000000,-6.700000,...,-6.300000,65.000000,8.200000,141.000000,-5.400000,62.000000,7.900000,163.000000,588.000000,2020.000000
50%,17531.500000,4.900000,74.000000,13.600000,228.000000,4.700000,74.000000,12.000000,230.000000,2.100000,...,2.700000,79.000000,11.900000,241.000000,4.300000,74.000000,11.800000,230.000000,1214.000000,2020.000000
75%,26297.250000,14.400000,84.000000,18.700000,274.000000,14.800000,84.000000,17.900000,292.000000,11.600000,...,12.300000,89.000000,16.400000,303.000000,14.200000,84.000000,16.600000,286.000000,1953.250000,2021.000000
max,35063.000000,33.600000,100.000000,57.300000,360.000000,33.400000,100.000000,59.300000,360.000000,31.800000,...,32.200000,100.000000,53.900000,360.000000,31.900000,100.000000,56.700000,360.000000,3504.000000,2022.000000


In [13]:
dayoutdf.describe()

,t_hours,temperature_2m_1,relative_humidity_2m_1,wind_speed_10m_1,wind_direction_10m_1,temperature_2m_33,relative_humidity_2m_33,wind_speed_10m_33,wind_direction_10m_33,temperature_2m_2,...,wind_direction_10m_31,temperature_2m_34,relative_humidity_2m_34,wind_speed_10m_34,wind_direction_10m_34,temperature_2m_38,relative_humidity_2m_38,wind_speed_10m_38,wind_direction_10m_38,Day_Ahead_Power
count,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,...,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35040.000000
mean,17531.500000,4.681411,73.201346,14.389191,207.287874,4.069296,72.305441,13.361519,216.556097,1.944556,...,226.443247,2.556263,76.076146,12.748138,218.991302,4.008750,72.276523,12.651241,218.051820,31280.918208
std,10122.249256,11.594230,14.859160,6.929509,88.373778,12.853228,14.483281,7.244159,86.048975,11.573345,...,90.769891,11.778780,16.236344,6.274645,95.029055,12.232864,15.215812,6.258694,86.492533,17077.945571
min,0.000000,-27.700000,23.000000,0.000000,1.000000,-34.000000,20.000000,0.000000,1.000000,-31.200000,...,1.000000,-31.400000,11.000000,0.000000,1.000000,-29.200000,21.000000,0.000000,1.000000,176.000000
25%,8765.750000,-4.100000,63.000000,9.300000,152.000000,-5.300000,62.000000,7.900000,140.000000,-6.700000,...,167.000000,-6.300000,65.000000,8.200000,141.000000,-5.400000,62.000000,7.900000,163.000000,17051.750000
50%,17531.500000,4.900000,74.000000,13.600000,228.000000,4.700000,74.000000,12.000000,230.000000,2.100000,...,245.000000,2.700000,79.000000,11.900000,241.000000,4.300000,74.000000,11.800000,230.000000,29891.000000
75%,26297.250000,14.400000,84.000000,18.700000,274.000000,14.800000,84.000000,17.900000,292.000000,11.600000,...,300.250000,12.300000,89.000000,16.400000,303.000000,14.200000,84.000000,16.600000,286.000000,44134.000000
max,35063.000000,33.600000,100.000000,57.300000,360.000000,33.400000,100.000000,59.300000,360.000000,31.800000,...,360.000000,32.200000,100.000000,53.900000,360.000000,31.900000,100.000000,56.700000,360.000000,79048.000000


## PCA

In [186]:
#let's do PCA on all the data
#onemonth = dayoutarray[0:30000]
#sing_val = np.linalg.svd(onemonth, compute_uv=False)
#print(sing_val)
total = np.sum(sing_val)
def percentage(v,n):
    total = np.sum(v)
    total_of_first_n = np.sum(v[:n])
    perc = total_of_first_n / total
    return float(perc)
percentage(sing_val,2)
#let's run a possibly weird series of calculations, randomly generate M intervals of I hours
#compute the percentage of singular values caputed by taking the first d singular values
#do this say 10 times
def average_dimensionality(A, M, I, d):
    percs = []
    for i in range(1, M+1):
        n = np.random.randint(1, 34341)
        B = A[n:n+I]
        v = np.linalg.svd(B, compute_uv=False)
        perc = percentage(v,d)
        percs.append(perc)
    return percs

def min_avg(A, I ,M, d):
    v = average_dimensionality(A, I, M, d)
    minim = np.min(v)
    return print(minim)

for i in range(1,50):
    l = []
    m = min_avg(dayoutarray, 100, 240, i)
    l.append(m)
    print(l)
    


0.6917999246860826
[None]
0.8594244380293363
[None]
0.9176441227076803
[None]
0.8880998171371955
[None]
0.8971328807802257
[None]
0.9211406792476043
[None]
0.9363023182656569
[None]
0.9308289118764571
[None]
0.9176388724407909
[None]
0.9445100908568715
[None]
0.9471219289855569
[None]
0.9259212401191486
[None]
0.9450916965083548
[None]
0.9558941519162997
[None]
0.9397689281186619
[None]
0.9636300392029594
[None]
0.9623648618253725
[None]
0.9635054533104389
[None]
0.9558946531681656
[None]
0.9588009987911585
[None]
0.9649021851638532
[None]
0.974457550451335
[None]
0.9748397247541988
[None]
0.964175009296305
[None]
0.9669631008098143
[None]
0.9744864758327064
[None]
0.9803405574358572
[None]
0.9786003744301264
[None]
0.9786015784261937
[None]
0.975637256967168
[None]
0.981799481564906
[None]
0.987259892264907
[None]
0.9854699857998577
[None]
0.9861281465123951
[None]
0.9855401818329652
[None]
0.988145830873749
[None]
0.9900363460035261
[None]
0.9885949550291975
[None]
0.9913547314256453

After running this a bit, this suggests that our data looks around 30-dimensional, which sort of makes sense, given that we have 39 wind farms and weather features probably correlate with eachother.

Let's just try out some models on this data.

## Linear Regression

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [15]:
#I need to remove some NaN entries from the matrix
dayoutarray = dayoutarray[0:35040,:]

dayoutarray[:,:157].shape

(35040, 157)

In [18]:
X= dayoutarray[:,1:157]
y= dayoutarray[:,157:158]

X_train = dayoutarray[0:26304,1:157] #train data from first 3 years
y_train= dayoutarray[0:26304,157:158]

X_test = dayoutarray[26304:35040,1:157] #test data from last four
y_test = dayoutarray[26304:35040,157:158]

model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

mse = mean_squared_error(y_test, pred)
r2 = r2_score(y_test, pred)

print(f"MSE: {mse:.2f}")
print(f"R2-score: {r2:.2f}")

MSE: 142916571.95
R2-score: 0.42


Not very good! Unsurprisingly.

In [19]:
y_train

array([[53069.],
       [53355.],
       [53663.],
       ...,
       [20600.],
       [20283.],
       [20113.]], shape=(26304, 1))

In [208]:
# we can compare the MSE with the total power generated during that year:
ogdf_trunc = copy.deepcopy(ogdf)
ogdf_trunc = ogdf_trunc.drop(columns=['time','Year', 'MonthDay'])
ogdf_trunc = ogdf_trunc.rename(columns={'Unnamed: 0': 't_hours'})
ogdf_trunc_array = ogdf_trunc.to_numpy()
v = ogdf_trunc_array[26304:35040,157:159]
(mse /(24 * np.sum(v)) )
#multiplying by 24 because we are looking one day out but measuring 24 tmes per day

np.float64(0.5308480163477728)

In the above cells I did one big line through the whole dataset, I could be a little more careful and see if weather data or say 1-30 days fits the next hour day well enough.

In [52]:
#if you want to try out regression for an interval of n hours, starting at time t0, and predicting F hours in the future do the following

n =24000 #length of time to train
t0 = 4000 #start time of training
F = 24 #hours in future
X_train = dayoutarray[t0:t0+n,1:157] #train data from first 10 days
y_train= dayoutarray[t0:t0+n,157:158]

X_test = dayoutarray[t0+n:t0+n+F,1:157] #test data for next 24 hours
y_test = dayoutarray[t0+n:t0+n+F,157:158]

model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

mse = mean_squared_error(y_test, pred)
r2 = r2_score(y_test, pred)

print(f"MSE: {mse:.2f}")
print(f"R2-score: {r2:.2f}")

MSE: 218791587.10
R2-score: -3.77


You might guess that looking at a small amount of data and then predicting the next day woul actually go well, but it seems that this is not the case! It is possible that this model works very poorly and the large dataset drowns out a lot of noise. Big n improves R2 score, t0 does not seem to (consistently) matter.

## Decision Tree

In [28]:
from sklearn.tree import DecisionTreeRegressor
#doingall the data feels crazy so I am just doing one month


X_train = dayoutarray[0:720,1:157] #train data from first 30 days
y_train= dayoutarray[0:720,157:158]

X_test = dayoutarray[720:744,1:157] #test data for next 24 hours
y_test = dayoutarray[720:744,157:158]


model = DecisionTreeRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)

mse = mean_squared_error(y_test, pred)
r2 = r2_score(y_test, pred)

print(f"MSE: {mse:.2f}")
print(f"R2-score: {r2:.2f}")

MSE: 1097498996.58
R2-score: -335.14


Very bad, maybe a similar issue to the situation above with linear regression.